In [1]:
import re
import time
import json
import jieba
import sklearn
import numpy as np
import pandas as pd
from collections import Counter
#加载专名发现后得到的词典
jieba.load_userdict( 'data/wesure_extra_dict.txt')

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\CAGITO~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.785 seconds.
Prefix dict has been built successfully.


In [2]:
#话术类词表加载
taolu_file = "data/stopwords_cn.txt"
taolu_list = [line.strip() for line in open(taolu_file, encoding='utf-8').readlines()]

In [3]:
import os
# 创建的目录
product_code = "Jan_KEEPER"
path = product_code + "_multigram_cluster_renew"
if not os.path.exists(path):
    os.mkdir(path)
    os.mkdir(path + '/cluster_2nd')
    os.mkdir(path + '/cluster_2nd_tags')

In [5]:
load_dict = []
with open('data/JAN_samples_isFAQ_with_1st_label_2nd_label_0322.txt','r',encoding='utf-8') as inf:
    for line in inf:
        data_json = json.loads(line.strip())
        if data_json["label_1st"] == "其他":
            load_dict.append(data_json)
            
print(len(load_dict))

142071


In [6]:
load_dict[0]

{'ws_user_id': '878da5995af3491f99ba0ff5926365bc',
 'userid': 'wmLpClCAAAjdWvPHboLxS0IWtoV0BkaA',
 'user_flag': 0,
 'user_name': '未来的.路',
 'session_time': '2021-01-19 14:14:45',
 'create_time': '2021-01-19 15:12:27',
 'direction': 0,
 'message_type': 'text',
 'is_faq': 0,
 'label_1st': '其他',
 'query': '[疑问]',
 'session_id': '210119ce6ac44fbe9af931befde5f20b'}

In [7]:
dict_clean = []
for dic in load_dict:
    temp = dic["query"]
    for word in taolu_list:
        temp = temp.replace(word,"")
    #过滤空白词与无意义的开头字
    temp = temp.strip(" ")
    temp = temp.strip("好")
    temp = temp.strip("的")
    temp = temp.strip("们")
    if len(temp) >= 3:
        dic["query_cleaned"] = temp
        dict_clean.append(dic)

In [8]:
#话术类词表加载
huashu_file = "data/huashu_words.txt"
huashu_list = [line.strip() for line in open(huashu_file, encoding='utf-8').readlines()]

In [9]:
bigram_dict = {}
trigram_dict = {}
multi_gram_clean_dict = []
for dic in dict_clean:
    sentence = dic["query_cleaned"]
    temp = list(jieba.lcut(sentence, cut_all=False))
    new_temp = [x for x in temp if (x != " " and x.isdigit() == False)]
    dic["unigram_cut"] = new_temp
    if len(new_temp) >= 3:
        bi_temp = []
        tri_temp = []
        for j in range(len(new_temp)-2):
            if new_temp[j] != new_temp[j+1]:
                tri = new_temp[j] + new_temp[j+1]+ new_temp[j+2]
                bi = new_temp[j] + new_temp[j+1]
                if (bi in bigram_dict.keys())==False:
                    bigram_dict[bi] = 1
                else:
                    bigram_dict[bi] += 1
                bi_temp.append(bi)
                if (tri in trigram_dict.keys())==False:
                    trigram_dict[tri] = 1
                else:
                    trigram_dict[tri] += 1
                tri_temp.append(tri)
        if new_temp[-2] != new_temp[-1]:
            bi_tail = new_temp[-2] + new_temp[-1]
            if (bi_tail in bigram_dict.keys())==False:
                bigram_dict[bi_tail] = 1
            else:
                bigram_dict[bi_tail] += 1
            bi_temp.append(bi_tail)
        bi_temp = list(set(bi_temp))
        tri_temp = list(set(tri_temp))
        dic["bigram_cut"] = bi_temp
        dic["trigram_cut"]  = tri_temp
        multi_gram_clean_dict.append(dic)
    elif len(new_temp) == 2:
        bi_temp = []
        temp_bi = new_temp[0] + new_temp[1]
        if (temp_bi in bigram_dict.keys())==False:
            bigram_dict[temp_bi] = 1
        else:
            bigram_dict[temp_bi] += 1
        bi_temp.append(temp_bi)
        dic["bigram_cut"]  = bi_temp
        dic["trigram_cut"]  = []
        multi_gram_clean_dict.append(dic)
    elif len(new_temp) == 1:
        dic["bigram_cut"]  = []
        dic["trigram_cut"]  = []
        multi_gram_clean_dict.append(dic)
    dic["unigram_cut"] = [x for x in new_temp if (x in huashu_list) == False]
        
print("Bigram all num:")
print(len(bigram_dict))
bigram_dict_top = {}
for word in bigram_dict:
    if bigram_dict[word] >= 10:
        bigram_dict_top[word] = bigram_dict[word]
print("Bigram top num:")
print(len(bigram_dict_top))
print("---------------------")
print("Trigram all num:")
print(len(trigram_dict))
trigram_dict_top = {}
for word in trigram_dict:
    if trigram_dict[word] >= 5:
        trigram_dict_top[word] = trigram_dict[word]
print("Trigram top num:")
print(len(trigram_dict_top))

Bigram all num:
144780
Bigram top num:
4170
---------------------
Trigram all num:
203255
Trigram top num:
3062


In [10]:
for dic in multi_gram_clean_dict:
    trigram_list = dic["trigram_cut"]
    trigram_top = []
    for tri in trigram_list:
        if tri in trigram_dict_top.keys():
            trigram_top.append(tri)
    bigram_list = dic["bigram_cut"]
    bigram_top = []
    for bi in bigram_list:
        if bi in bigram_dict_top.keys():
            bigram_top.append(bi)
    unigram_list = dic["unigram_cut"]
    multigram = unigram_list+bigram_top+trigram_top
    dic["multigram_cut"] = " ".join(multigram) 
        
multi_gram_list = []
for dic in multi_gram_clean_dict:
    multi_gram_list.append(dic["multigram_cut"])
print(multi_gram_list[0:5])

['给 孩子 买 孩子买 给孩子 给孩子买', '什么 保险 什么 骗人 什么的 保险什么 什么保险', '有 什么 有什么', '哪 种 基本 重疾 哪种', '看 下 我看 看下 我看下']


In [11]:
multi_gram_clean_dict[0]

{'ws_user_id': '60f7086757d943978470dbd314dc9199',
 'userid': 'wmLpClCAAAJQ-ixjll-P0_rkozXhGD3g',
 'user_flag': 0,
 'user_name': '许盼亮',
 'session_time': '2021-01-18 17:27:08',
 'create_time': '2021-01-18 18:10:49',
 'direction': 0,
 'message_type': 'text',
 'is_faq': 1,
 'label_1st': '其他',
 'query': '我只是想一直给孩子买',
 'session_id': '2101185390f1498c99d989f2ba54087c',
 'query_cleaned': '给孩子买',
 'unigram_cut': ['给', '孩子', '买'],
 'bigram_cut': ['孩子买', '给孩子'],
 'trigram_cut': ['给孩子买'],
 'multigram_cut': '给 孩子 买 孩子买 给孩子 给孩子买'}

In [14]:
multi_gram_words = []
multi_gram_time = []
multi_gram_user_id = []
multi_gram_user_name = []
multi_gram_ws_user_id = []
multi_gram_words_clean = []
multi_gram_session_id = []
for dic in multi_gram_clean_dict:
    multi_gram_words.append(dic["query"])
    multi_gram_time.append(dic["create_time"])
    multi_gram_user_id.append(dic["userid"])
    multi_gram_user_name.append(dic["user_name"])
    multi_gram_ws_user_id.append(dic["ws_user_id"])
    multi_gram_words_clean.append(dic["query_cleaned"])
    multi_gram_session_id.append(dic["session_id"])

In [11]:
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
#将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer = CountVectorizer()
#该类会统计每个词语的tf-idf权值
transformer = TfidfTransformer()
#第chat_clean一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵
tfidf = transformer.fit_transform(vectorizer.fit_transform(multi_gram_list))
word = vectorizer.get_feature_names()

from sklearn.cluster import KMeans

start = time.process_time() 
clu_2000, n_clusters = KMeans(n_clusters=17).fit(tfidf), 17
end = time.process_time() 
print('Running time: %s Seconds'%(end-start))

Running time: 7785.328125 Seconds


In [15]:
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
#将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer = CountVectorizer()
#该类会统计每个词语的tf-idf权值
transformer = TfidfTransformer()
#第chat_clean一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵
tfidf = transformer.fit_transform(vectorizer.fit_transform(multi_gram_list))
word = vectorizer.get_feature_names()

from sklearn.cluster import KMeans

def train_cluster(train_vecs, start_k, end_k):
    SSE = []
    SSE_d1 = [] #sse的一阶导数
    SSE_d2 = [] #sse的二阶导数
    models = [] #保存每次的模型
    for i in range(start_k, end_k):
        kmeans_model = KMeans(n_clusters=i)
        kmeans_model.fit(train_vecs)
        SSE.append(kmeans_model.inertia_)  # 保存每一个k值的SSE值
        #print('{} Means SSE loss = {}'.format(i, kmeans_model.inertia_))
        models.append(kmeans_model)
    # 求二阶导数，通过sse方法计算最佳k值
    SSE_length = len(SSE)
    for i in range(1, SSE_length):
        SSE_d1.append((SSE[i - 1] - SSE[i]) / 2)
    for i in range(1, len(SSE_d1) - 1):
        SSE_d2.append((SSE_d1[i - 1] - SSE_d1[i]) / 2)

    best_model = models[SSE_d2.index(max(SSE_d2)) + 1]
    print("Choice of i:" + str(SSE_d2.index(max(SSE_d2)) + 1 + start_k))
    k_choice = SSE_d2.index(max(SSE_d2)) + 1 + start_k
    return best_model, k_choice

start = time.process_time() 
clu_2000, n_clusters = train_cluster(tfidf, start_k=16, end_k=20)
end = time.process_time() 
print('Running time: %s Seconds'%(end-start))

Choice of i:17
Running time: 4601.875 Seconds


In [16]:
key_choice = 20
order_centroid = clu_2000.cluster_centers_.argsort()[:, ::-1]
cluster_20=[]
for i in range(n_clusters):
    clusters=[]
    for ind in order_centroid[i, :key_choice]:
        clusters.append(word[ind])
    cluster_20.append(clusters)
    
cluster_20_df = pd.DataFrame(cluster_20)
cluster_20_df.to_excel(path + "/"+product_code+"1st_cluster_"+str(n_clusters)+"_cluster_tags.xlsx" , index=True)

cluster_bag = []
for line in cluster_20:
    for word in line:
        cluster_bag.append(word)
        
cluster_bag = list(set(cluster_bag))
cluster_bag.sort(key=lambda x:len(x),reverse=True)

cluster_bag = list(set(cluster_bag))
cluster_bag.sort(key=lambda x:len(x),reverse=True)

result_file = path + "/cluster_"+str(n_clusters)+"_cluster_tags_bag.txt"
output = open(result_file, 'w', encoding="utf-8")
for row in cluster_bag:
    output.write(row)
    output.write('\n')
output.close()

In [14]:
multi_gram_words = []
multi_gram_time = []
multi_gram_user_id = []
multi_gram_user_name = []
multi_gram_ws_user_id = []
multi_gram_words_clean = []
multi_gram_session_id = []
for dic in multi_gram_clean_dict:
    multi_gram_words.append(dic["query"])
    multi_gram_time.append(dic["create_time"])
    multi_gram_user_id.append(dic["userid"])
    multi_gram_user_name.append(dic["user_name"])
    multi_gram_ws_user_id.append(dic["ws_user_id"])
    multi_gram_words_clean.append(dic["query_cleaned"])
    multi_gram_session_id.append(dic["session_id"])

In [18]:
chat_clean_cluster = pd.DataFrame()
chat_clean_cluster['cluster_id'] = clu_2000.labels_
chat_clean_cluster["query"] = multi_gram_words
chat_clean_cluster["query_cleaned"] = multi_gram_words_clean
chat_clean_cluster["query_cut"] = multi_gram_list
chat_clean_cluster["time"] = multi_gram_time
chat_clean_cluster["user_id"] = multi_gram_user_id
chat_clean_cluster["user_name"] = multi_gram_user_name
chat_clean_cluster["ws_user_id"] = multi_gram_ws_user_id
chat_clean_cluster["session_id"] = multi_gram_session_id
chat_clean_cluster = chat_clean_cluster.sort_values(by = 'cluster_id',axis = 0,ascending = True)
chat_clean_cluster = chat_clean_cluster.reset_index(drop=True)
chat_clean_cluster.head()

,cluster_id,query,query_cleaned,query_cut,time,user_id,user_name,ws_user_id,session_id
0,0,我只是想一直给孩子买,给孩子买,给 孩子 买 孩子买 给孩子 给孩子买,2021-01-18 18:10:49,wmLpClCAAAJQ-ixjll-P0_rkozXhGD3g,许盼亮,60f7086757d943978470dbd314dc9199,2101185390f1498c99d989f2ba54087c
1,0,这个比较精,比较精,比较 精,2021-01-06 14:56:03,wmLpClCAAAeQqRmdEdOHhrY3ARAnGwNA,lotus,8e00466899744411beb83471646f5c7d,210106aeb55b42d8ac4447c648a0c1a5
2,0,要休息半年才能看到咋样的！,要休息半年咋样,要 休息 半年,2021-01-18 14:54:33,wmLpClCAAARxNfWJIaR9bd6mh7cURX5Q,热爱生活,b930507f35ed4b949fda2f83da33bada,210118ded15b47379ec62457ccc37cdc
3,0,有糖尿病,有糖尿病,有 糖尿病 有糖尿病,2021-01-05 12:08:41,wmLpClCAAAnnP4GYyVKZBMEztnD0aFng,相见恨晚,ab2e00135a31423daf288157e203c15b,2101052a2840404fbbec8a5d3723e51c
4,0,微信聊吧,微信聊,微信 聊 微信聊,2021-01-08 10:56:48,wmLpClCAAAwsfbRgB90_FGMaOywZ_HiA,阿丽丝韵皮肤修复云云,8b4df27ecb79451b8d58377bd1c56eec,2101088a45e6418f986724c6301b3b8c


In [19]:
labels=[]
for i in range(len(chat_clean_cluster)):
    label_temp = []
    cluster_label = int(chat_clean_cluster["cluster_id"][i])
    for word in cluster_20[cluster_label]:
        if word in str(chat_clean_cluster["query_cut"][i]):
            label_temp.append(word)
    labels.append(" ".join(label_temp))

chat_clean_cluster["label_keys"] = labels

chat_clean_cluster = chat_clean_cluster[["cluster_id", "label_keys", "query", "query_cleaned",\
                                         "query_cut", "time", "session_id", "user_id", "user_name","ws_user_id"]]

chat_clean_cluster.to_excel(path + "/" + product_code + "1st_cluster_result_"+str(n_clusters)+".xlsx" , index=False)

In [21]:
cluster_tag = []
for line in cluster_20:
    cluster_tag.append(line[0])
    
chat_cluster_count = {}
for i in range(len(chat_clean_cluster)):
    cluster_id_temp = chat_clean_cluster["cluster_id"][i]
    if (cluster_id_temp in chat_cluster_count.keys()) == False:
        chat_cluster_count[cluster_id_temp] = 1
    else:
        chat_cluster_count[cluster_id_temp] += 1
        
chat_cluster_count_list = []
for i in range(len(chat_cluster_count)):
    count = chat_cluster_count[i]
    chat_cluster_count_list.append(count)
    
extra_cluster = chat_cluster_count_list.index(max(chat_cluster_count_list))

cluster_2nd_results = pd.DataFrame()

In [22]:
extra_cluster

0

In [24]:
cluster_2nd_results = pd.DataFrame()

In [25]:
for i in range(n_clusters):
    chat_cluster_temp=chat_clean_cluster.loc[chat_clean_cluster['cluster_id']==i]
    chat_cluster_temp=chat_cluster_temp.reset_index(drop=True)
    if len(chat_cluster_temp) > 20:
        user_chat_cut0 = list(chat_cluster_temp["query_cut"])
        tfidf_temp = transformer.fit_transform(vectorizer.fit_transform(user_chat_cut0))
        word = vectorizer.get_feature_names()
        if i == extra_cluster:
            sub_clu_2000, k = KMeans(n_clusters=17).fit(tfidf_temp), 17
        else:
            start_k=5
            end_k = 12
            sub_clu_2000, k = train_cluster(tfidf_temp, start_k, end_k)
        chat_cluster_temp['cluster_id_2nd'] = sub_clu_2000.labels_
        chat_cluster_temp = chat_cluster_temp.sort_values(by = 'cluster_id_2nd', axis = 0, ascending=True)
        chat_cluster_temp = chat_cluster_temp.reset_index(drop=True)
        sub_n_clusters = k
        if i == extra_cluster:
            key_choice = 20
        else:
            key_choice = 10
        order_centroids = sub_clu_2000.cluster_centers_.argsort()[:, ::-1]
        cluster_sub=[]
        for j in range(sub_n_clusters):
            clusters=[]
            for ind in order_centroids[j, :key_choice]:
                clusters.append(word[ind])
            cluster_sub.append(clusters)
        #对于杂项聚类需要给出聚类簇标签        
        sub_cluster_20_df = pd.DataFrame(cluster_sub)
        if i == extra_cluster:
            sub_cluster_20_df.to_excel(path +"/cluster_2nd_tags/"+product_code+"extra_cluster_"+str(i)+"_"+str(k)+"_cluster_tags.xlsx", index=True)
        else:
            sub_cluster_20_df.to_excel(path +"/cluster_2nd_tags/"+product_code+"2nd_cluster_"+str(i)+"_"+str(k)+"_cluster_tags.xlsx", index=True)
        cluster_sub_bag = []
        for j in range(len(cluster_sub)):
            for word in cluster_sub[j]:
                cluster_sub_bag.append(word)
        cluster_sub_bag = list(set(cluster_sub_bag))
        cluster_sub_bag.sort(key=lambda x:len(x),reverse=True)
        sub_labels=[]
        for j in range(len(chat_cluster_temp)):
            label_temp = []
            cluster_label = int(chat_cluster_temp["cluster_id_2nd"][j])
            for word in cluster_sub[cluster_label]:
                if word in str(chat_cluster_temp["query_cut"][j]):
                    label_temp.append(word)
            sub_labels.append(" ".join(label_temp).replace("健告知","健康告知"))
        chat_cluster_temp["label_keys_2nd"] = sub_labels
        chat_cluster_temp = chat_cluster_temp[['cluster_id', "label_keys","cluster_id_2nd", "label_keys_2nd",  "query", "query_cleaned",\
                                         "query_cut", "time", "session_id", "user_id", "user_name","ws_user_id"]]
        cluster_2nd_results = cluster_2nd_results.append(chat_cluster_temp)
        if i != extra_cluster:
            chat_cluster_temp.to_excel(path+"/cluster_2nd/2nd_recluster_"+str(i)+"_"+str(sub_n_clusters)+".xlsx" , index=False)
        else:
            chat_cluster_temp.to_excel(path+"/cluster_2nd/2nd_extra_recluster_"+str(i)+"_"+str(sub_n_clusters)+".xlsx", index=False)
        print("Cluster: " + str(i) + " Recluster k: " + str(k))
    else:
        print("Too short to re-cluster: " + str(i))
    if i == extra_cluster:
        chat_cluster_count = {}
        for j in range(len(chat_cluster_temp)):
            cluster_id_temp = chat_cluster_temp["cluster_id_2nd"][j]
            if (cluster_id_temp in chat_cluster_count.keys()) == False:
                chat_cluster_count[cluster_id_temp] = 1
            else:
                chat_cluster_count[cluster_id_temp] += 1
        chat_cluster_count_list = []
        for j in range(k):
            count = chat_cluster_count[j]
            chat_cluster_count_list.append(count)
        extra_cluster_temp = chat_cluster_count_list.index(max(chat_cluster_count_list))
        print("----------------------------------------------")
        print("to be third cluster:")
        recluster_index = i
        print(str(recluster_index)+"_"+str(extra_cluster_temp))
        print("----------------------------------------------")

cluster_2nd_results.to_excel(path + "/"+product_code+"_2nd_cluster_result_"+str(n_clusters)+".xlsx" , index=False)

Cluster: 0 Recluster k: 17
----------------------------------------------
to be third cluster:
0_10
----------------------------------------------
Choice of i:9
Cluster: 1 Recluster k: 9
Choice of i:8
Cluster: 2 Recluster k: 8
Choice of i:6
Cluster: 3 Recluster k: 6
Choice of i:8
Cluster: 4 Recluster k: 8
Choice of i:8
Cluster: 5 Recluster k: 8
Choice of i:7
Cluster: 6 Recluster k: 7
Choice of i:8
Cluster: 7 Recluster k: 8
Choice of i:7
Cluster: 8 Recluster k: 7
Choice of i:9
Cluster: 9 Recluster k: 9
Choice of i:8
Cluster: 10 Recluster k: 8
Choice of i:9
Cluster: 11 Recluster k: 9
Choice of i:6
Cluster: 12 Recluster k: 6
Choice of i:9
Cluster: 13 Recluster k: 9
Choice of i:8
Cluster: 14 Recluster k: 8
Choice of i:7
Cluster: 15 Recluster k: 7
Choice of i:6
Cluster: 16 Recluster k: 6


In [26]:
df = cluster_2nd_results
df.head()

,cluster_id,label_keys,cluster_id_2nd,label_keys_2nd,query,query_cleaned,query_cut,time,session_id,user_id,user_name,ws_user_id
0,0,联系 我有,0,联系 联系你 我有,那先给我看一下吧，后面我有疑问再联系你,那先看后面我有疑问联系你,那先 看 后面 有 疑问 联系 后面我 我有 有疑问 联系你,2021-01-05 17:58:44,2101056c2b8647c7a126479ec98a6255,wmLpClCAAARsgxS7rI2BB08VDe37zUNA,元姐姐,20a5a5d26ef94435890a0317ccfa2a30
1,0,时间,0,时间 有时间,你在忙吗，有时间和我聊好吗,你在忙有时间和我聊,在 忙 有 时间 聊 忙有 在忙 和我 有时间 你在 在忙有 忙有时间,2021-01-08 18:27:04,2101084c528a44dd93a36f984d45240c,wmLpClCAAAqEdNBy_DV_5eEDdcb6pO6A,丁庭武 天然蜂蜜,80276cae493d4bee9321ac5fc1138770
2,0,联系,0,联系,好的[OK] 尽量中午12-1点左右联系。谢谢,尽量中午121点左右联系,尽量 中午 点 左右 联系 点左右 中午点,2021-01-15 15:53:52,2101152a734249a9ab259e5799999242,wmLpClCAAA7okM61tk8oev5u4kO5LbPg,AA优速快递 (UC优速),d24ddf5d5f024c1483adc1dfc9eeffb9
3,0,时间,0,时间 没时间,现在没时间,没时间,没 时间 没时间,2021-01-15 17:33:39,2101151c7876496a878d55e8730ab13b,wmLpClCAAASmk_LBt5XCX37TVNCmDo4A,lk,5e3af798120349468f5b37af8457f46b
4,0,联系,0,联系 联系你,以后有毛病了，我就联系你吗,有毛病我就联系你,有 毛病 联系 就联系 我就 联系你 就联系你 我就联系,2021-01-15 10:00:08,21011595679e4d149190c2c9534bc440,wmLpClCAAAC20HzBYOSjODEh-332yaDQ,不要打扰我,3ac4b9a91d2046b6aece251e5b103e17


In [27]:
labels_new = []
labels_new_set = []
label1 = list(df["cluster_id"])
label2 = list(df["cluster_id_2nd"])
for i in range(len(label1)):
    new_label = str(label1[i]) + "-" + str(label2[i])
    if (new_label in labels_new_set) == False:
        labels_new_set.append(new_label)
    labels_new.append(new_label)
print(len(labels_new))    
df["cluster_new"] = labels_new

103810


In [28]:
from collections import Counter

word_labels = []
for i in range(len(labels_new_set)):
    chat_cluster_temp=df.loc[df['cluster_new']== labels_new_set[i]]
    chat_cluster_temp=chat_cluster_temp.reset_index(drop=True)
    cut_list = list(chat_cluster_temp["query_cut"])
    cut_list = [str(x).split(" ") for x in cut_list]
    word_bag = []
    for line in cut_list:
        for word in line:
            if len(word) >= 2:
                word_bag.append(word)
    counter = Counter(word_bag)
    word_counts_top = counter.most_common(20)
    labels = [x[0] for x in word_counts_top]
    word_labels.append(labels)

In [21]:
from gensim.models import word2vec
from gensim.models import KeyedVectors
model = KeyedVectors.load("models/w2v.model", mmap='r')
vocab = list(model.wv.vocab.keys())

In [22]:
def vec_average_ranked(vecs, dim):
    vec_result = []
    for i in range(dim):
        summ = 0
        #加入词频排序信息
        for j in range(len(vecs)):
            summ += vecs[j][i] * (len(vecs) - j)
        avg = (2 * summ) / (len(vecs) * (len(vecs) + 1))
        vec_result.append(avg)
    return vec_result

def vec_average(vecs, dim):
    vec_result = []
    for i in range(dim):
        summ = 0
        #加入词频排序信息
        for j in range(len(vecs)):
            summ += vecs[j][i]
        avg = summ / len(vecs) 
        vec_result.append(avg)
    return vec_result

In [23]:
vec_labels = []
for line in word_labels:
    vecs = []
    for word in line:
        if word in vocab:
            vec = model[word.replace("\n","")]
            vecs.append(vec)
    vec_line = vec_average(vecs, 200)
    vec_labels.append(vec_line)
    
print(len(vec_labels))

<ipython-input-23-5467a73d48b1>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec = model[word.replace("\n","")]


128


In [24]:
def train_cluster(train_vecs, start_k, end_k):
    SSE = []
    SSE_d1 = [] #sse的一阶导数
    SSE_d2 = [] #sse的二阶导数
    models = [] #保存每次的模型
    for i in range(start_k, end_k):
        kmeans_model = KMeans(n_clusters=i)
        kmeans_model.fit(train_vecs)
        SSE.append(kmeans_model.inertia_)  # 保存每一个k值的SSE值
        #print('{} Means SSE loss = {}'.format(i, kmeans_model.inertia_))
        models.append(kmeans_model)
    # 求二阶导数，通过sse方法计算最佳k值
    SSE_length = len(SSE)
    for i in range(1, SSE_length):
        SSE_d1.append((SSE[i - 1] - SSE[i]) / 2)
    for i in range(1, len(SSE_d1) - 1):
        SSE_d2.append((SSE_d1[i - 1] - SSE_d1[i]) / 2)

    best_model = models[SSE_d2.index(max(SSE_d2)) + 1]
    print("Choice of i:" + str(SSE_d2.index(max(SSE_d2)) + 1 + start_k))
    k_choice = SSE_d2.index(max(SSE_d2)) + 1 + start_k
    return best_model, k_choice

start = time.process_time() 
clu_2000, n_clusters = train_cluster(vec_labels, start_k = 25, end_k = 55)
end = time.process_time() 
print('Running time: %s Seconds'%(end-start))

Choice of i:31
Running time: 4.015625 Seconds


In [25]:
chat_clean_cluster = pd.DataFrame()
chat_clean_cluster['cluster_id_4clusters'] = clu_2000.labels_
chat_clean_cluster['cluster_id_original'] = labels_new_set
chat_clean_cluster["word"] = [" ".join(x) for x in word_labels]
chat_clean_cluster = chat_clean_cluster.sort_values(by = 'cluster_id_4clusters',axis = 0,ascending = True)
chat_clean_cluster = chat_clean_cluster.reset_index(drop=True)
chat_clean_cluster.head()

,cluster_id_4clusters,cluster_id_original,word
0,0,1-4,百万医疗险 退百万医疗险
1,1,15-1,不想 不想买
2,1,15-2,不想 不想买 投保 不想投保 续费 不想续费 不想投 不想续 不想入 不想上 不想保 不想交...
3,1,15-4,不想 不想保
4,1,15-5,不想 不想交 原因


In [26]:
chat_clean_cluster.to_excel(path + "/"+product_code+"_二次聚类_高频词表_重聚类结果_词向量平均.xlsx" , index=False)